In [1]:
import pandas as pd
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

C:\Users\Gumpun\AppData\Local\Temp\ipykernel_10028\2925815428.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def file2list(fname):
    with open(fname, encoding='utf-8') as fp:  # Specify the encoding
        lines = fp.readlines()
        return [line.strip() for line in lines]

In [3]:
pos_text = file2list("Downloads/asp_pos (1).txt")
neg_text = file2list("Downloads/asp_neg.txt")

In [4]:
pos_df = pd.DataFrame(pos_text,columns=['review'])
pos_df['label'] = "positive"

neg_df = pd.DataFrame(neg_text,columns=['review'])
neg_df['label'] = "negative"
neg_df

data = pd.concat([pos_df, neg_df])
data.sample(100)

,review,label
76,ส่งเร็วนะ ค่อนข้างชอบ เสียอย่างเดียวสีตก เกือบ...,negative
502,ชอบมากครับ จัดส่งเร็ว ปกติใส่37 แต่สั่ง36 พอดี...,positive
149,ส่งสินค้าไว แพคมาอย่างดี สีกับแบบตามรูปเลย แต่...,positive
142,จัดส่งช้ามากมาก เลทกว่าที่เขียนไว้ 2-3 วัน ไม่...,negative
177,ก็โอเค ทรงสวยดีแต่ใส่ไม่ถนัด ชาร์จไม่เข้าไม่ดีเลย,negative
...,...,...
160,ของถึงไวมาก แต่ติดหยุดยาวเลยกดรับช้าค่ะ ปกติใส...,positive
443,สินค้าดีมีคุณภาพและมาตรฐาน quick silver ใส่สวย...,positive
106,สินค้าคุณภาพดีเยี่ยมไม่ผิดหวังเลยในการใช้งาน ห...,positive
588,หลังอ่านรีวิวเลยตัดสินใจกดซื้อช่วงแฟรชเซลล์+ไล...,positive


In [5]:
category_counts = data['label'].value_counts()
category_counts

label
positive    630
negative    235
Name: count, dtype: int64

In [6]:
!pip install pythainlp

In [7]:
from pythainlp.tokenize import word_tokenize
import re

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def thai_clean_text(text):
    st = ""

    text = deEmojify(text)
    text = text.replace("\n"," ")
    for w in word_tokenize(text):
        st = st + w + " "

    return  re.sub(' +', ' ', st)

data['review'] = data.review.apply(thai_clean_text)
data.sample(5)

,review,label
44,สินค้า แย่มาก ถัง ปั่น แกะ ออกมา มี รอย ปิ่น ไ...,negative
176,สินค้า ใช้ ระยะเวลา จัดส่ง นาน มาก ๆ ค่ะ เพราะ...,negative
550,สินค้า สวย คุณภาพดี แถม ถุงเท้า ให้ 1 คู่,positive
11,ใช้งาน ดี ใส่ ได้ พอดี จัดส่ง ไว ราคา ถูก ด้วย...,positive
74,ส่ง ไว ดี สินค้า เป็น ของ ใหม่ และ ของแท้ การข...,positive


In [8]:
reviews = data['review'].values
labels = data['label'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [9]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify=encoded_labels, test_size=0.3, random_state=42)

In [10]:
from tqdm import tqdm
vec = CountVectorizer(max_features = 1000)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names_out()
X = X.toarray()
word_counts = {}

#ใช้ข้อมูล  2 class
for l in range(2):
    word_counts[l] = defaultdict(lambda: 0)
for i in tqdm(range(X.shape[0])):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]

100%|██████████| 605/605 [00:00<00:00, 1983.60it/s]


In [11]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]

    #print(data)
    return data

def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():

        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors

def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}

        words = set(text.strip().split(" ")) #split text with space
        #print(words)
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        #print(max(label_scores, key=label_scores.get))
        result.append(max(label_scores, key=label_scores.get))
    return result

In [12]:
labels = [0,1]
n_label_items, log_label_priors = fit(train_sentences,train_labels,labels)

In [13]:
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred))

Accuracy of prediction on test set :  0.7269230769230769


In [14]:
test_text = thai_clean_text("สินค้า ถูกต้อง ตามที่ สั่ง ถูก ต้องตาม")
predicted_label = predict(n_label_items, vocab, word_counts, log_label_priors, labels, [test_text])

if predicted_label[0] == 0:
    print("The text is classified as negative.")
else:
    print("The text is classified as positive.")

The text is classified as positive.


In [15]:
tokenized_text = word_tokenize(thai_clean_text(test_text), keep_whitespace=False)
tokenized_text

['สินค้า', 'ถูกต้อง', 'ตามที่', 'สั่ง', 'ถูก', 'ต้องตาม']

In [16]:
for i in tokenized_text:
    print(i)
    test_text = thai_clean_text(i)
    sentiment = predict(n_label_items, vocab, word_counts, log_label_priors, labels, [test_text])
    print(f"Predicted sentiment: {sentiment}")
    print("----------------------")

สินค้า
Predicted sentiment: [1]
----------------------
ถูกต้อง
Predicted sentiment: [1]
----------------------
ตามที่
Predicted sentiment: [1]
----------------------
สั่ง
Predicted sentiment: [1]
----------------------
ถูก
Predicted sentiment: [1]
----------------------
ต้องตาม
Predicted sentiment: [1]
----------------------


In [17]:
from sklearn.metrics import classification_report
print(classification_report(test_labels,pred))

              precision    recall  f1-score   support

           0       0.50      0.01      0.03        71
           1       0.73      0.99      0.84       189

    accuracy                           0.73       260
   macro avg       0.61      0.50      0.43       260
weighted avg       0.67      0.73      0.62       260

